In [410]:
# allocate_when_greater function allocates the given without checking any condition and is used as a tool to fire up when 
# required conditions are made in makes changes to the quantity and thus returns it 
# since all lists are dynamically made there is an auto change in the list .
# z is the allocatio finally decided to be stored for location i as per the sizes 
def allocate_when_greater(quantity_in_kgs, sizes, logistics_constraints,topups, available_capacity,i,lt,rem):
                z=0
                lt.append(i)
                available_capacity[i] = available_capacity[i] -logistics_constraints[i]
                z=z+logistics_constraints[i]
                quantity_in_kgs =quantity_in_kgs-logistics_constraints[i]*sizes[i]
                while quantity_in_kgs > topups[i]*sizes[i] and available_capacity[i] >=topups[i] :
                        quantity_in_kgs =quantity_in_kgs-(topups[i]*sizes[i])
                        available_capacity[i] = available_capacity[i] -topups[i]
                        z=z+topups[i] 
                rem.append(z) 
                return (quantity_in_kgs)

In [411]:
# it checks up conditions and fires up allocate_when_greater function
# rem is a list of alloctions makes which is mapped to indices which is stored in lt irrespective of whether or not it is allocated or not
# to allocate the capacity available must be greater than logistic constraints atleast
# if the remaining quantity is greater than the minimum weight to be allocated then we fire up allocate_when_greater
# else for deallocating prior indice we check which function which tells us which is the most recent indice allocated such that whose deallocation can create pathway for allocation of ith indice
# if no such indice exists indice == -1 , else wise we just proceed to feasible function and print the hash map using print_hash_map
def allocate(quantity_in_kgs, sizes, logistics_constraints,topups, available_capacity):
    rem=[]
    lt=[]
    for i in range(0,len(available_capacity)):
        if logistics_constraints[i] <= available_capacity[i] :
            if quantity_in_kgs >= logistics_constraints[i]*sizes[i]:
                quantity_in_kgs=allocate_when_greater(quantity_in_kgs, sizes, logistics_constraints,topups, available_capacity,i,lt,rem)
            else:
                 m=0
                 indice = which(quantity_in_kgs, sizes, logistics_constraints,topups, available_capacity,lt,rem)
                 k = lt[indice]
                 if indice != -1:
                     (lt,rem,quantity_in_kgs,available_capacity) = feasible(quantity_in_kgs, sizes, logistics_constraints,topups, available_capacity,k,i,rem,m,lt)
                     print_hash_map("reminder",quantity_in_kgs,"variant",sizes,"uoms",
                                            logistics_constraints,"allocations",rem,"post_alloc_capacity",
                                            available_capacity)
                     return
        else:
            lt.append(i)
            rem.append(0)        
    print_hash_map("reminder",quantity_in_kgs,"variant",sizes,"uoms",
                        logistics_constraints,"allocations",rem,"post_alloc_capacity",
                        available_capacity)
    return

In [412]:
# return the indice which can be deallocated in response o the requirement to allocate ith indice 
def which(quantity_in_kgs, sizes, logistics_constraints,topups, available_capacity,lt,rem):
    for i in range(len(lt)-1,-1,-1):
        if rem[i] > logistics_constraints[i]:
            return i
    return -1

In [413]:
# returns the quantity and constrain_fullfill_i whether or not changes can be made or not ,
# it proceeds system to next allocated phase if possible
# or retains the same phase if not possible
def edit(quantity_in_kgs, sizes, logistics_constraints,topups,available_capacity ,k,i,rem,m,lt,constrain_to_fullfill_i):
    dumy_quantity_in_kgs =quantity_in_kgs
    dumy_constrain_to_fullfill_i = constrain_to_fullfill_i
    for j in range(k,-1,-1):
        while constrain_to_fullfill_i > 0 and (rem[j]) > logistics_constraints[j]:
            constrain_to_fullfill_i = constrain_to_fullfill_i - (topups[j]*sizes[j])
            available_capacity[j]=available_capacity[j]+topups[j]
            rem[j]=rem[j]-topups[j]
            quantity_in_kgs = quantity_in_kgs + (topups[j]*sizes[j])
        if constrain_to_fullfill_i <=0:
            return (quantity_in_kgs , constrain_to_fullfill_i )
        if j==0:
            return(dumy_quantity_in_kgs , dumy_constrain_to_fullfill_i)  

In [414]:
#copies the initial elements 
# proceed to next phase using edit
# if most feasible , profitable and reminder less achieved then remain in here
# if not return back ie copy back to original seen constraints
def feasible(quantity_in_kgs, sizes, logistics_constraints,topups,available_capacity ,k,i,rem,m,lt):
    sz=sizes.copy()
    lc=logistics_constraints.copy()
    tps=topups.copy()
    ac=available_capacity.copy()
    rm=rem.copy()
    l=lt.copy()
    check_rem=quantity_in_kgs
    check_rem1 =abs(quantity_in_kgs - logistics_constraints[i]*sizes[i])
    constrain_to_fullfill_i = available_capacity[i]*sizes[i] - quantity_in_kgs
    (quantity_in_kgs ,constrain_to_fullfill_i )=edit(quantity_in_kgs, sizes, logistics_constraints,topups,available_capacity ,k,i,rem,m,lt,constrain_to_fullfill_i)
    if abs(constrain_to_fullfill_i) < check_rem and abs(constrain_to_fullfill_i) < check_rem1 :
        if quantity_in_kgs >= logistics_constraints[i]*sizes[i]:
            quantity_in_kgs=allocate_when_greater(quantity_in_kgs, sizes, logistics_constraints,topups, available_capacity,i,lt,rem)
            return(lt,rem,quantity_in_kgs,available_capacity)
    elif check_rem1 < abs(constrain_to_fullfill_i) and check_rem1 < check_rem:
        sizes=sz.copy()
        logistics_constraints=lc.copy()
        topups=tps.copy()
        available_capacity=ac.copy()
        rem=rm.copy()
        lt=l.copy()
        quantity_in_kgs=check_rem
        if quantity_in_kgs < logistics_constraints[i]*sizes[i]:  
                quantity_in_kgs=allocate_when_greater(quantity_in_kgs, sizes, logistics_constraints,topups, available_capacity,i,lt,rem)
                return(lt,rem,quantity_in_kgs,available_capacity)
    elif check_rem < abs(constrain_to_fullfill_i) and check_rem < check_rem1 :
        sizes=sz.copy()
        logistics_constraints=lc.copy()
        topups=tps.copy()
        available_capacity=ac.copy()
        rem=rm.copy()
        lt=l.copy()
        quantity_in_kgs=check_rem
        return(lt,rem,quantity_in_kgs,available_capacity)

In [415]:
#prints a hash map using dictionary datastructure in python 
def print_hash_map(reminder,reminder_value,variant,variant_value,uoms,uoms_value,allocation,allocation_value,post_allocation,post_allocation_value):
    dict1 = {
        reminder : reminder_value,
        variant : variant_value,
        uoms : uoms_value,
        allocation : allocation_value,
        post_allocation : post_allocation_value
    }
    print(dict1)

In [432]:
# driver function to fireup all the test cases 
def driver():
    weight=[123,146,289,457,280]
    size = [0.5, 1, 2, 5]
    variant = [[50,50,50,30],[50,50,50,20],[50,50,50,20],[50,50,50,20],[50,50,50,20]]
    topups = [10,10,10,10]
    allocation =[[63,125,40,20] , [103,15,27,20] , [60,65,55,20] , [60,65,55,20] , [65,65,65,20]]
    for i in range(0,len(weight)):
        allocate(weight[i],size,variant[i],topups,allocation[i])
    
if __name__ == '__main__':
    driver()

{'reminder': 3.0, 'variant': [0.5, 1, 2, 5], 'uoms': [50, 50, 50, 30], 'allocations': [60, 90, 0, 0], 'post_alloc_capacity': [3, 35, 40, 20]}
{'reminder': 1.0, 'variant': [0.5, 1, 2, 5], 'uoms': [50, 50, 50, 20], 'allocations': [90, 0, 0, 20], 'post_alloc_capacity': [13, 15, 27, 0]}
{'reminder': -1.0, 'variant': [0.5, 1, 2, 5], 'uoms': [50, 50, 50, 20], 'allocations': [60, 60, 50, 20], 'post_alloc_capacity': [0, 5, 5, 0]}
{'reminder': 167.0, 'variant': [0.5, 1, 2, 5], 'uoms': [50, 50, 50, 20], 'allocations': [60, 60, 50, 20], 'post_alloc_capacity': [0, 5, 5, 0]}
{'reminder': 0.0, 'variant': [0.5, 1, 2, 5], 'uoms': [50, 50, 50, 20], 'allocations': [60, 50, 50, 20], 'post_alloc_capacity': [5, 15, 15, 0]}
